In [ ]:
using Pkg; Pkg.activate(".")
using Dates

In [1]:
nheader = 1
ncolumn = 7

"read gps file GPRMC data"
function read_gps_data(filename;
    nheader=nheader,
    nsample=countlines(joinpath(mastdir,string(yday(dt)),filename)) - nheader,
    ncolumn=ncolumn )

    # preallocate the data
    psltime = Vector{String}(undef, nsample) # will point to data as it is read
    gpsdt = status = lat = lon = speed = course = Vector{Float64}(undef, nsample)

    open(joinpath(mastdir,string(yday(dt)),filename)) do file
        for _ in 1:nheader
            readline(file) # skip header
        end
        nl = 0
        for line in readlines(file)
            fields = split(line, [' ','\t'])
            if startswith(fields[2], "\$GPRC")
                nl += 1
                psltime[nl] = fields[1] # string
                gpsdt[nl], status[nl], lat[nl], lon[nl], speed[nl], course[nl] = parse_gpmrc(fields[2])
            end
        end
    end
    
    psldt = psldatetime.(dt, Hour(parse(Int32, filename[9:10])), psltime)

    # Return a dictionary with the parsed fields
    return Dict(
        "datetime" => psldt[1:nl],
        "status" => status[1:nl],
        "latitude" => lat[1:nl],
        "longitude" => lon[1:nl],
        "speed_knots" => speed[1:nl],
        "course_degrees" => course[1:nl] )
end

"yearday from Date or DateTime"
yday(dt) = Dates.value(Date(dt) - Date(year(dt)-1,12,31))

"Convert a PSL date to a DateTime."
function psldatetime(dt, hh, psltime)
    mm  = map(t -> Minute(     parse(Int32, t[1:2])), psltime)
    SS  = map(t -> Second(     parse(Int32, t[3:4])), psltime)
    sss = map(t -> Millisecond(parse(Int32, t[5:7])), psltime)
    datatime =  DateTime(dt) + Hour(parse(Int32, filename[9:10])) + mm + SS + sss
end

"""
Parse a GPRMC NMEA message.

Arguments:
- `message::String`: GPRMC NMEA message

Returns:
- A dictionary containing the parsed fields
"""
function parse_gprmc(message::String)
    # Split the message into fields
    fields = split(message, ",")
    
    # Check that the message is a GPRMC message
    # and position is valid.
    if fields[1] != "\$GPRMC"
        error("Not a GPRMC message")
    elseif fields[3] != "A"
        error("Invalid position status")
    end
    
    # Extract the fields
    time_str = fields[2]
    # status = fields[3]
    lat_str = fields[4]
    lat_dir = fields[5]
    lon_str = fields[6]
    lon_dir = fields[7]
    speed_str = fields[8]
    course_str = fields[9]
    date_str = fields[10]
    
    # Parse the time (hhmmss.sss) and date (ddmmyy)
    time = tryparse(Time, time_str[1:6], dateformat"HHMMSS")
    date = tryparse(Date, date_str, dateformat"ddmmyy")

    # Combine date and time into a DateTime object
    gpstime = try DateTime(date, time) catch error("Invalid date or time format")

    # Parse latitude and longitude in degrees and minutes (ddmm.mmmm)
    lat_deg = tryparse(Float64, lat_str[1:2])
    lat_min = tryparse(Float64, lat_str[3:end])
    lat = lat_deg + lat_min / 60
    if lat_dir == "S"
        lat = -lat
    end

    lon_deg = tryparse(Float64, lon_str[1:3])
    lon_min = tryparse(Float64, lon_str[4:end])
    lon = lon_deg + lon_min / 60
    if lon_dir == "W"
        lon = -lon
    end

    # Parse speed (knots) and course (degrees)
    speed = tryparse(Float64, speed_str)
    course = tryparse(Float64, course_str)

    # Return a dictionary with the parsed fields
    return gpstime, status, lat, lon, speed, course
end

# Example usage
message = "\$GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W*6A"
parsed_data = parse_gprmc(message)
println(parsed_data)



LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///Users/deszoeks/Projects/ASTRAL/mast/In[1]#88:46\[90mIn[1]:88:46[0;0m]8;;\
    # Combine date and time into a DateTime object
    datetime = try DateTime(date, time) catch[48;2;120;70;70m error("Invalid date or time format")[0;0m
[90m#                                            └───────────────────────────────────┘ ── [0;0m[91ma variable name is expected after `catch`[0;0m